# Introduction to kikuchipy

## NNUM in Oslo, Norway June 4, 2024

In this tutorial we will characterize a transmission Kikuchi diffraction (TKD) dataset of polycrystalline gold by Hough indexing and pattern matching.
The results will be evaluated by using geometrical TKD simulations and comparing indexing results to pre-indexing maps.

The dataset is gracefully provided by Alice Bastos da Silva from the Technical University of Denmark (DTU).

Steps:

1. Load and inspect data
2. Specify candidate phases
3. Calibrate detector-sample geometry (projection/pattern center)
4. Hough indexing
5. Dictionary indexing
6. Orientation refinement

Documentation of tools we use:

* kikuchipy: https://kikuchipy.org
* HyperSpy: https://hyperspy.org
* PyEBSDIndex: https://pyebsdindex.readthedocs.io
* orix: https://orix.readthedocs.io
* diffsims: https://diffsims.readthedocs.io
* EMsoft (indirectly): https://github.com/EMsoft-org/EMsoft

Import all necessary functionality

In [132]:
# Replace 'inline' for 'qt5' (from PyQt5 package) for interactive plotting
%matplotlib qt5

from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

from diffsims.crystallography import ReciprocalLatticeVector
import hyperspy.api as hs
import kikuchipy as kp
from orix import io, plot, sampling
from orix.crystal_map import Phase, PhaseList
from orix.quaternion import Orientation, Rotation
from orix.vector import Vector3d


#plt.rcParams.update(
#    {"figure.facecolor": "w", "font.size": 15, "figure.dpi": 100}
#)

## 1. Load and inspect data

Load data (or download to cache for the first time; subsequent calls will fetch the cached data)

In [2]:
path_data = Path("/Users/hakon/data/ebsd/alice_fanta_tkd/2")
path_res = path_data / "results"

In [3]:
path_tkd = path_data / "20nmAu_2019-03-12_02.hdf5"
s = kp.load(path_tkd, lazy=False)

In [4]:
s

<EBSD, title: 20nmAu_2019-03-12_02..., dimensions: (51, 61|320, 240)>

In [5]:
s.metadata

├── Acquisition_instrument
│   └── SEM
│       ├── beam_energy = 30.0
│       ├── magnification = 100000.0
│       └── working_distance = 5.026599722
├── General
│   ├── original_filename = W:\P87500-Alice\Alice2019\PenumbraFebMarch2019\20nmAu_std\Au20_Std_30KV_1-7nA_ap30_Sp5_DD5_DT5-8_HV_dyn__400p.bcf
│   └── title = 20nmAu_2019-03-12_02...
└── Signal
    └── signal_type = EBSD

In [6]:
s.original_metadata

├── CameraTilt = 5.824999998357347
├── Coordinate Systems
│   ├── ESPRIT Coordinates = array([[254, 254, 254, ..., 254, 254, 254],
       [254, 254, 254, ..., 254, 2 ... ..., 254, 254, 254],
       [254, 254, 254, ..., 254, 254, 254]], dtype=uint8)
│   └── ID = 5
├── DetectorFullHeightMicrons = 23700
├── DetectorFullWidthMicrons = 31600
├── Grid Type = isometric
├── KV = 30.0
├── MADMax = 1.8
├── Magnification = 100000.0
├── MapStepFactor = 1.0
├── MaxRadonBandCount = 12
├── MinIndexedBands = 5
├── NCOLS = 299
├── NPoints = 3111
├── NROWS = 258
├── OriginalFile = W:\P87500-Alice\Alice2019\PenumbraFebMarch2019\20nmAu_std\Au20_Std_30KV_1-7nA_ap30_Sp5_DD5_DT5-8_HV_dyn__400p.bcf
├── PatternHeight = 240
├── PatternWidth = 320
├── Phases
│   └── 1
│       ├── AtomPositions
│       │   └── 1 = Au,0,0,0,1,0
│       ├── Formula = Au4
│       ├── IT = 225
│       ├── LatticeConstants = array([ 4.079,  4.079,  4.079, 90.   , 90.   , 90.   ], dtype=float32)
│       ├── Name = Gold
│       ├── Setting = 1
│       └── SpaceGroup = F m#ovl3m
├── PixelByteCount = 2
├── SEPixelSizeX = 0.005002701667
├── SEPixelSizeY = 0.005083576806
├── SampleTilt = 0.0
├── StaticBackground = array([[167, 154, 157, ..., 132, 140, 145],
       [162, 165, 163, ..., 146, 1 ... .., 126, 138, 136],
       [148, 150, 153, ..., 138, 128, 128]], dtype=uint16)
├── TopClip = 0.0
├── UnClippedPatternHeight = 240
├── WD = 5.026599722
├── XSTEP = 0.005002701667
├── YSTEP = 0.005083576806
├── ZOffset = 0.0
├── manufacturer = bruker nano
└── version = esprit 2.x

In [7]:
s.detector

EBSDDetector (240, 320), px_size 98.75 um, binning 1, tilt 5.824999998357347, azimuthal 0, pc (0.489, 0.439, -0.63)

In [8]:
s.detector.sample_tilt = 90.0

In [9]:
xmap = io.load(path_tkd)
xmap

Phase   Orientations         Name  Space group  Point group  Proper point group     Color
   -1     284 (9.1%)  not_indexed         None         None                None         w
    1   2827 (90.9%)         Gold        Fm-3m         m-3m                 432  tab:blue
Properties: PCX, PCY, DD, MAD, MADPhase, NIndexedBands, RadonBandCount, RadonQuality, XBEAM, YBEAM, XSAMPLE, YSAMPLE, ZSAMPLE
Scan unit: um

### Some pre-indexing maps

Before indexing, it is important to:
* Evaluate the pattern quality
* Get an impression of what to expect from the indexing results

To that end, we inspect three different pre-indexing maps:
* Mean pattern intensity map
* Virtual backscatter electron (VBSE) imaging RGB map
* Image quality map

Get the mean pattern intensity map

In [10]:
s_mean = s.mean(axis=(2, 3))
s_mean

<BaseSignal, title: 20nmAu_2019-03-12_02..., dimensions: (51, 61|)>

Inspect patterns in this map

In [11]:
s.plot(s_mean)

Save mean pattern intensity map

In [12]:
plt.imsave(path_res / "maps_mean.png", s_mean.data, cmap="gray")

Set up VBSE image generator

In [13]:
vbse_imager = kp.imaging.VirtualBSEImager(s)
vbse_imager.grid_shape = (5, 5)
vbse_imager

VirtualBSEImager for <EBSD, title: 20nmAu_2019-03-12_02..., dimensions: (51, 61|320, 240)>

Plot VBSE grid

In [14]:
vbse_imager.plot_grid()

<EBSD, title: 20nmAu_2019-03-12_02..., dimensions: (|320, 240)>

Specify RGB colour channels

In [15]:
r = [(1, 1), (2, 1), (3, 1)]
b = [(1, 2), (3, 2)]
g = [(1, 3), (2, 3), (3, 3)]

Plot coloured grid tiles

In [16]:
vbse_imager.plot_grid(rgb_channels=[r, g, b])

<EBSD, title: 20nmAu_2019-03-12_02..., dimensions: (|320, 240)>

Get VBSE RGB image

In [17]:
vbse_rgb = vbse_imager.get_rgb_image(r, g, b)

In [18]:
vbse_rgb.plot()

In [22]:
s.plot(vbse_rgb)

Save RGB image

In [19]:
vbse_rgb.save(path_res / "maps_vbse_rgb.png", overwrite=True)

### Enhancement of Kikuchi pattern

Remove static (constant) background

In [20]:
bg = s.mean(axis=(0, 1))
bg.change_dtype(s.data.dtype)
bg

<EBSD, title: 20nmAu_2019-03-12_02..., dimensions: (|320, 240)>

In [21]:
s.remove_static_background(operation="divide", static_bg=bg.data)

[########################################] | 100% Completed | 211.66 ms


Inspect a few (computed on the fly) statically corrected patterns

In [22]:
s.plot(s_mean)

Save corrected patterns

In [ ]:
#s.save("patterns.h5")

Create pattern mask

In [109]:
origin_shift = [-3, -2]
r_inner, r_outer = 35, 160

origin = np.array(s.detector.shape) // 2
origin += origin_shift

signal_mask = np.ones(s.detector.shape, dtype=bool)
dist = kp.filters.distance_to_origin(signal_mask.shape, origin=origin)
signal_mask[dist < r_inner] = False
signal_mask[dist > r_outer] = False
(s.inav[0, 30] * signal_mask).plot(vmin=10_000)

Inspect masked patterns

In [110]:
(s * signal_mask).plot(s_mean, vmin=10_000)

### More pre-indexing maps

Get image quality $Q$ map (not image quality IQ from Hough indexing!)

In [25]:
maps_iq = s.get_image_quality()

[########################################] | 100% Completed | 531.45 ms


Navigate patterns in $Q$ map

In [26]:
s.plot(hs.signals.Signal2D(maps_iq), vmin=10_000)

Save $Q$ map

In [27]:
plt.imsave(path_res / "maps_iq.png", maps_iq, cmap="gray")

## 2. Specify candidate phases

We do not expect other phases than Ni.
We will load the master pattern of Ni (packaged with kikuchipy) created with EMsoft

In [28]:
path_mp = "/Users/hakon/data/crystals/au/au_mc_mp_tkd_30kv.h5"
mp = kp.load(path_mp, energy=30, projection="lambert")

In [29]:
mp.phase.name = "au"

Create a phase list for use in Hough indexing, and set the lattice parameters to Ångström

In [30]:
pl = PhaseList(mp.phase)
lat = pl["au"].structure.lattice
lat.setLatPar(lat.a * 10, lat.b * 10, lat.c * 10)
pl

Id  Name  Space group  Point group  Proper point group     Color
 0    au        Fm-3m         m-3m                 432  tab:blue

Also, load the stereographic projection which we will plot later to understand a bit more about simulations

In [31]:
mp_sp = kp.load(path_mp, energy=30, projection="stereographic")

## 3. Calibrate detector-sample geometry

Inspect calibration read from the vendor

In [32]:
s.detector.sample_tilt = 90.0

Orientation of detector with respect to the sample:
* Known:
    * Sample tilt (about microscope X) = 6 degrees
    * No camera tilt (screen horizontal)
* Unknown:
    * Projection/pattern center (PCx, PCy, PCz): Shortest distance from source point to detector

Find suitable calibration patterns to get a mean PC for the data

In [33]:
s.plot(hs.signals.Signal2D(maps_iq))

In [34]:
xy = [
    (14, 0),
    (10, 12),
    (17, 32),
    (15, 53),
    (37, 51),
    (42, 37),
    (42, 22),
]
p_cal = np.zeros((len(xy),) + s.detector.shape, dtype=s.data.dtype)
for i, (x, y) in enumerate(xy):
    p_cal[i] = s.data[y, x]
s_cal = kp.signals.EBSD(p_cal, detector=s.detector)

Plot pattern positions in an overview image (specific to NORDIF acquisition software)

In [35]:
rc = np.array(xy)[:, ::-1]

kp.draw.plot_pattern_positions_in_map(
    rc=rc,
    roi_shape=maps_iq.shape,
    roi_image=maps_iq,
    color="r",
)

Inspect calibration patterns

In [36]:
(s_cal * signal_mask).plot(navigator="none")

Generate an indexer instance used to optimize the PC and to perform Hough indexing

In [37]:
indexer_cal = s_cal.detector.get_indexer(pl)

In [38]:
g = ReciprocalLatticeVector(pl["au"], hkl=[[1, 1, 1], [2, 0, 0], [2, 2, 0], [3, 1, 1]])
g = g.symmetrise()
g.sanitise_phase()  # Complete unit cell
g.calculate_structure_factor()
g.calculate_theta(30e3)
g.print_table()

 h k l      d     |F|_hkl   |F|^2   |F|^2_rel   Mult 
 1 1 1    2.356    27.0     727.6     100.0      8   
 2 0 0    2.040    24.5     599.5      82.4      6   
 2 2 0    1.442    18.3     334.9      46.0      12  
 3 1 1    1.230    15.5     241.3      33.2      24  


Create simulator

In [39]:
simulator = kp.simulations.KikuchiPatternSimulator(g)

Plot simulator

In [40]:
simulator.plot(mode="bands", axes_labels=["e1", "e2"])

Inspect default PC with gnomonic circles with 10$^{\circ}$ spacing

In [41]:
s_cal.detector.plot("gnomonic", draw_gnomonic_circles=True)

In [42]:
s_cal2 = s_cal.deepcopy()
s_cal2.rescale_intensity(percentiles=(0.1, 99.9))

[########################################] | 100% Completed | 106.42 ms


Optimize PC by trial and error (the initial guess is based on previous experiments on the same microscope).

NB! Running this cell the first time after installing PyEBSDIndex might cause a kernel crash...
If that happens, you have to re-run the notebook.
It should not happen again.
This is an issue we're working to solve.

In [43]:
det_cal = s_cal.hough_indexing_optimize_pc([0.49, 0.44, -0.63], indexer_cal, batch=True, method="PSO")

# Print PCs and standard deviations
print("All PCs:\n", det_cal.pc)
print("Mean PC:\n", det_cal.pc_average)
print("Std:\n", det_cal.pc.std(axis=0), "\n")

# Get new indexer from detector, using the average PC
indexer_cal2 = det_cal.get_indexer(pl)

# Index calibration patterns using found PCs
xmap_cal = s_cal.hough_indexing(pl, indexer_cal2, verbose=0)
print("\n", xmap_cal, "\n")

# Create geometrical simulations for each pattern
sim_cal = simulator.on_detector(det_cal, xmap_cal.rotations)


PC found: [********* ] 7/7  global best:0.1  PC opt:[ 0.4829  0.4332 -0.6382]49]
All PCs:
 [[ 0.48082426  0.42406403 -0.62835403]
 [ 0.48046304  0.43162949 -0.63328057]
 [ 0.48704533  0.43106702 -0.62952331]
 [ 0.4776462   0.43058224 -0.63052349]
 [ 0.49695517  0.41683134 -0.62341703]
 [ 0.48199189  0.42021414 -0.62491618]
 [ 0.48290195  0.43316551 -0.63816217]]
Mean PC:
 [ 0.48397541  0.42679339 -0.62973954]
Std:
 [0.00591983 0.00593524 0.00461828] 

Hough indexing with PyEBSDIndex information:
  PyOpenCL: True
  Projection center (Bruker, mean): (0.484, 0.4268, -0.6297)
  Indexing 7 pattern(s) in 1 chunk(s)
  Indexing speed: 96.81727 patterns/s

 Phase  Orientations  Name  Space group  Point group  Proper point group     Color
    0    7 (100.0%)    au        Fm-3m         m-3m                 432  tab:blue
Properties: fit, cm, pq, nmatch
Scan unit: px 

Finding bands that are in some pattern:
[########################################] | 100% Completed | 105.81 ms
Finding zone axes 

Plot patterns with markers

In [44]:
#del s_cal2.metadata.Markers
markers = sim_cal.as_markers(lines_kwargs={"linewidth": 0.5, "color": "r"})
s_cal2.add_marker(markers, plot_marker=False, permanent=True)
(s_cal2 * signal_mask).plot(None, vmin=10_000)

Plot pair of PC coordinates and compare to the map of PC positions from above

In [45]:
det_cal.plot_pc("scatter", annotate=True)

Refine PCs using pattern matching

In [47]:
xmap_cal2, det_cal2 = s_cal.refine_orientation_projection_center(
    xmap_cal,
    det_cal,
    mp,
    energy=30,
    signal_mask=~signal_mask,
    method="LN_NELDERMEAD",
    trust_region=[5, 5, 5, 0.1, 0.1, 0.1],
    rtol=1e-5,
    chunk_kwargs=dict(chunk_shape=1),  # Spread across all CPUs
)

# Create geometrical simulations for each pattern
sim_cal2 = simulator.on_detector(det_cal2, xmap_cal2.rotations)

Refinement information:
  Method: LN_NELDERMEAD (local) from NLopt
  Trust region (+/-): [5.  5.  5.  0.1 0.1 0.1]
  Relative tolerance: 1e-05
Refining 7 orientation(s) and projection center(s):
[####################################    ] | 91% Completed | 108.10 ms

/opt/homebrew/Caskroom/miniforge/base/envs/kp/lib/python3.10/site-packages/kikuchipy/signals/util/_crystal_map.py:41: UserWarning: Signal navigation axes must be named 'x' and/or 'y' in order to compare the signal navigation scales to the crystal map step sizes 'dx' and 'dy' (see `EBSD.axes_manager`)
  warnings.warn(


[########################################] | 100% Completed | 1.16 sms
Refinement speed: 6.02826 patterns/s
Finding bands that are in some pattern:
[########################################] | 100% Completed | 105.88 ms
Finding zone axes that are in some pattern:
[########################################] | 100% Completed | 101.30 ms
Calculating detector coordinates for bands and zone axes:
[########################################] | 100% Completed | 106.59 ms


Again, plot patterns with markers

In [48]:
#del s_cal2.metadata.Markers
markers2 = sim_cal2.as_markers(lines_kwargs={"linewidth": 0.5, "color": "w"})
s_cal2.add_marker(markers2, plot_marker=False, permanent=True)

In [49]:
(s_cal2 * signal_mask).plot(None, vmin=10_000)

In [50]:
det_cal2.plot_pc("scatter", annotate=True)

Save detector

In [51]:
det_cal2.save(path_res / "det_cal.txt")

Get detector with mean PC

In [52]:
det1 = det_cal2.deepcopy()
det1.pc = det1.pc_average

## 4. Hough indexing

Create a new indexer with the extrapolated PCs

In [53]:
indexer = det1.get_indexer(pl)

Hough index all patterns

In [54]:
xmap_hi = s.hough_indexing(pl, indexer, verbose=2)

Hough indexing with PyEBSDIndex information:
  PyOpenCL: True
  Projection center (Bruker): (0.4826, 0.4281, -0.6241)
  Indexing 3111 pattern(s) in 6 chunk(s)
Radon Time: 0.49512137717101723
Convolution Time: 0.14714791695587337
Peak ID Time: 0.2757090841187164
Band Label Time: 0.47608129133004695
Total Band Find Time: 1.3944448749534786
Band Vote Time:  0.7949642080347985
  Indexing speed: 1404.09427 patterns/s


Check whether any patterns could not be indexed

In [55]:
xmap_hi

Phase   Orientations         Name  Space group  Point group  Proper point group     Color
   -1      17 (0.5%)  not_indexed         None         None                None         w
    0   3094 (99.5%)           au        Fm-3m         m-3m                 432  tab:blue
Properties: fit, cm, pq, nmatch
Scan unit: um

Save the Hough indexing results

In [56]:
io.save(path_res / "xmap_hi.h5", xmap_hi, overwrite=True)

Plot the pattern fit and confidence metric (CM) maps

In [57]:
xmap_hi.plot("fit", colorbar=True, colorbar_label="fit", vmax=3)

In [58]:
xmap_hi.plot("cm", colorbar=True, colorbar_label="cm")

In [59]:
xmap_hi.plot("pq", colorbar=True, colorbar_label="pq")

In [60]:
xmap_hi.plot("nmatch", colorbar=True, colorbar_label="nmatch")

Plot the orientation map (IPF-Z), where colors are given by the (symmetry reduced) crystal direction $\left<uvw\right>$ pointing in the into-plane direction

In [61]:
ckey = plot.IPFColorKeyTSL(xmap_hi.phases[0].point_group)

In [59]:
ckey.plot()

In [62]:
rgb_hi = ckey.orientation2color(xmap_hi.rotations)
xmap_hi.plot(rgb_hi)

Plot IPF-Z map overlayed with image quality

In [63]:
xmap_hi.plot(rgb_hi, overlay=maps_iq.ravel())

In [64]:
xmap_hi.plot(rgb_hi, overlay="pq")

Evaluate results by plotting geometrical simulations on top of patterns

In [65]:
sim_hi = simulator.on_detector(det1, xmap_hi.rotations.reshape(*xmap_hi.shape))

Finding bands that are in some pattern:
[########################################] | 100% Completed | 106.30 ms
Finding zone axes that are in some pattern:
[########################################] | 100% Completed | 106.50 ms
Calculating detector coordinates for bands and zone axes:
[########################################] | 100% Completed | 105.41 ms


In [66]:
s.axes_manager["dx"].scale = s.axes_manager["dy"].scale = 1.0

In [67]:
#del s.metadata.Markers  # Uncomment to delete previously added markers
s.add_marker(sim_hi.as_markers(lines_kwargs={"linewidth": 0.5}), permanent=True, plot_marker=False)

Create a navigator from the IPF-Z map

In [68]:
maps_ipf = kp.draw.get_rgb_navigator(rgb_hi.reshape(xmap_hi.shape + (3,)))

In [69]:
s.plot(maps_ipf)

## 5. Dictionary indexing

Dictionary indexing is more robust towards noise in EBSD patterns (resulting from e.g. overlapping bands, high degree of deformation etc.) than Hough indexing.
A dictionary consists of a series of (dynamically) simulated patterns projected from a master pattern, and we compare all experimental patterns to all these.

To create this dictionary, we need:
* Accurate detector-sample geometry (already done!)
* Master pattern (in the square Lambert projection)
* Sampling of all possible orientations (per phase)

Plot our geometrical simulation on top of the upper stereographic projection of our dynamically simulated Ni Kikuchi sphere

In [70]:
fig, ax = plt.subplots(subplot_kw={"projection": "stereographic"}, figsize=(7, 7))
simulator.plot(mode="bands", color="w", figure=fig)
ax.imshow(mp_sp.data, cmap="gray", extent=(-1, 1, -1, 1));

Discretely sample the complete orientation space of point group $m\bar{3}m$ (*Oh*) with an average misorientation of about 2$^{\circ}$ between rotations $\mathbf{g}$

In [71]:
R_sample = sampling.get_sample_fundamental(
    resolution=2, point_group=mp.phase.point_group
)
O_sample = Orientation(R_sample, symmetry=mp.phase.point_group)

In [72]:
O_sample

Orientation (100347,) m-3m
[[ 0.8541 -0.3536 -0.3536 -0.1435]
 [ 0.8541 -0.3536 -0.3536  0.1435]
 [ 0.8541 -0.3536 -0.1435 -0.3536]
 ...
 [ 0.8541  0.3536  0.1435  0.3536]
 [ 0.8541  0.3536  0.3536 -0.1435]
 [ 0.8541  0.3536  0.3536  0.1435]]

Plot a subset of sampled orientations in axis-angle space

In [73]:
O_sample.get_random_sample(2000).scatter()

Bin patterns

In [74]:
s2 = s.downsample(4, inplace=False)

[########################################] | 100% Completed | 105.99 ms


In [75]:
s2.detector.shape = s2.detector.shape[::-1]

In [76]:
det2 = det1.deepcopy()
det2.shape = s2.axes_manager.signal_shape[::-1]

Set up generation of the dictionary of dynamically simulated patterns

In [77]:
s_dict = mp.get_patterns(O_sample, det2, energy=30, chunk_shape=5000)
s_dict

<LazyEBSD, title: , dimensions: (100347|80, 60)>

Create new pattern mask

In [138]:
del s2.metadata.Markers

In [112]:
bin_factor = 4
origin_shift = [-1, -1]
r_inner, r_outer = 35 // bin_factor, 160 // bin_factor

origin = np.array(s2.detector.shape) // 2
origin += origin_shift

signal_mask2 = np.ones(s2.detector.shape, dtype=bool)
dist = kp.filters.distance_to_origin(signal_mask2.shape, origin=origin)
signal_mask2[dist < r_inner] = False
signal_mask2[dist > r_outer] = False
(s2.inav[0, 30] * signal_mask2).plot(vmin=10_000)

Inspect the five first patterns in the dictionary

In [91]:
fig = plt.figure(figsize=(20, 4), layout="tight")
hs.plot.plot_images((s_dict.inav[:5] * signal_mask2), axes_decor=None, per_row=5, fig=fig, colorbar=False);

Perform dictionary indexing by generating a chunk of simulated patterns at a time and compare them to all the experimental patterns

In [92]:
xmap_di = s2.dictionary_indexing(
    s_dict,
    signal_mask=~signal_mask2,
)

Dictionary indexing information:
  Phase name: au
  Matching 3111 experimental pattern(s) to 100347 dictionary pattern(s)
  NormalizedCrossCorrelationMetric: float32, greater is better, rechunk: False, navigation mask: False, signal mask: True


100%|███████████████████████████████████████████████████████████████████| 21/21 [00:20<00:00,  1.01it/s]


  Indexing speed: 149.61538 patterns/s, 15013454.53375 comparisons/s


In [93]:
xmap_di

Phase   Orientations  Name  Space group  Point group  Proper point group     Color
    0  3111 (100.0%)    au        Fm-3m         m-3m                 432  tab:blue
Properties: scores, simulation_indices
Scan unit: um

The 20 best matches (rotations, scores and simulation indices) are kept

In [94]:
xmap_di.scores.shape

(3111, 20)

Plot similarity scores (normalized cross-correlation, NCC) between best matching
experimental and simulated patterns

In [95]:
xmap_di.plot(xmap_di.scores[:, 0], colorbar=True, colorbar_label="NCC")

Plot IPF-Z orientation map

In [96]:
rgb_di = ckey.orientation2color(xmap_di.rotations[:, 0])
xmap_di.plot(rgb_di)

Plot IPF-Z orientation map with scores

In [97]:
xmap_di.plot(rgb_di, overlay=xmap_di.scores[:, 0])

Save dictionary indexing results to file

In [98]:
io.save(path_res / "xmap_di.h5", xmap_di, overwrite=True)

## 6. Orientation refinement

During refinement, a better score $r$ is searched for iteratively by changing the orientation (and/or PC) slightly in a controlled manner using an optimization algorithm.
The default algorithm is the Nelder-Mead simplex from SciPy.
We here use that from the NLopt package (an optional dependency of kikuchipy), which has been found to be faster but equally accurate.

In [114]:
xmap_ref = s.refine_orientation(
    xmap=xmap_di,
    detector=det1,
    master_pattern=mp,
    energy=30,
    signal_mask=~signal_mask,
    method="LN_NELDERMEAD",
    trust_region=[2, 2, 2],
)

Refinement information:
  Method: LN_NELDERMEAD (local) from NLopt
  Trust region (+/-): [2 2 2]
  Relative tolerance: 0.0001
Refining 3111 orientation(s):
[########################################] | 100% Completed | 69.64 ss
Refinement speed: 44.66838 patterns/s


In [115]:
xmap_ref

Phase   Orientations  Name  Space group  Point group  Proper point group     Color
    0  3111 (100.0%)    au        Fm-3m         m-3m                 432  tab:blue
Properties: scores, num_evals
Scan unit: um

Plot refined orientation map

In [144]:
ckey.direction = Vector3d([0, 0, 1])

In [145]:
rgb_ref = ckey.orientation2color(xmap_ref.rotations)
xmap_ref.plot(rgb_ref)

Plot refined NCC scores

In [117]:
xmap_ref.plot(xmap_ref.scores, colorbar=True, colorbar_label="NCC")

Plot orientation map with scores overlayed

In [118]:
xmap_ref.plot(rgb_ref, overlay="scores")

Compare histogram of scores after DI and refinement

In [119]:
fig, ax = plt.subplots()
ax.hist(xmap_di.scores[:, 0], bins=100, color="C0", label="DI", alpha=0.5)
ax.hist(xmap_ref.scores, bins=100, color="C1", label="Ref", alpha=0.5)
ax.legend()
ax.set(xlabel="NCC", ylabel="Frequency");

Save final indexing results

In [120]:
io.save(path_res / "xmap_ref.ang", xmap_ref, overwrite=True)
io.save(path_res / "xmap_ref.h5", xmap_ref, overwrite=True)

Compare to simulations

In [126]:
sim_ref = simulator.on_detector(det1, xmap_ref.rotations.reshape(*xmap_ref.shape))

Finding bands that are in some pattern:
[########################################] | 100% Completed | 106.13 ms
Finding zone axes that are in some pattern:
[########################################] | 100% Completed | 106.89 ms
Calculating detector coordinates for bands and zone axes:
[########################################] | 100% Completed | 107.14 ms


In [66]:
s.axes_manager["dx"].scale = s.axes_manager["dy"].scale = 1.0

In [159]:
sim_ref.plot()

IndexError: list index out of range

In [157]:
markers = sim_ref.as_markers(
    zone_axes_labels=True,
    lines_kwargs={"linewidth": 0.5},
)

In [158]:
markers

[<marker.LineSegment, line_segment (x1=nan,x2=nan,y1=nan,y2=nan,color=r)>,
 <marker.LineSegment, line_segment (x1=174.72677102474395,x2=174.72677102474395,y1=174.72677102474395,y2=174.72677102474395,color=r)>,
 <marker.LineSegment, line_segment (x1=nan,x2=nan,y1=nan,y2=nan,color=r)>,
 <marker.LineSegment, line_segment (x1=353.62550985309576,x2=353.62550985309576,y1=353.62550985309576,y2=353.62550985309576,color=r)>,
 <marker.LineSegment, line_segment (x1=nan,x2=nan,y1=nan,y2=nan,color=r)>,
 <marker.LineSegment, line_segment (x1=nan,x2=nan,y1=nan,y2=nan,color=r)>,
 <marker.LineSegment, line_segment (x1=nan,x2=nan,y1=nan,y2=nan,color=r)>,
 <marker.LineSegment, line_segment (x1=33.68077657767036,x2=33.68077657767036,y1=33.68077657767036,y2=33.68077657767036,color=r)>,
 <marker.LineSegment, line_segment (x1=nan,x2=nan,y1=nan,y2=nan,color=r)>,
 <marker.LineSegment, line_segment (x1=nan,x2=nan,y1=nan,y2=nan,color=r)>,
 <marker.LineSegment, line_segment (x1=-47.345124975256475,x2=-47.34512497

In [148]:
del s.metadata.Markers  # Uncomment to delete previously added markers
s.add_marker(markers, permanent=True, plot_marker=False)

Create a navigator from the IPF-Z map

In [151]:
maps_ipf = kp.draw.get_rgb_navigator(rgb_ref.reshape(xmap_ref.shape + (3,)))

In [152]:
s.plot(maps_ipf)